# plotting quantified data

In [ ]:
import xarray as xr
import pint_xarray

## load the data

In [ ]:
ds = xr.tutorial.open_dataset("air_temperature")
data = ds.air
data

## convert units into a format understood by pint

<!-- Use one of the alert classes: alert-info, alert-warning, alert-success, alert-danger -->
<div class="alert alert-info">
    <b>Note:</b> this example uses the data provided by the <tt>xarray.tutorial</tt> functions. As such, the units attributes follow the CF conventions, which by default <tt>pint</tt> does not understand. To work around that, we're modifying the <tt>units</tt> attributes here, but in general it is better to use a library that modifies the <tt>UnitRegistry</tt>.
</div>

In [ ]:
data.lat.attrs["units"] = "degree"
data.lon.attrs["units"] = "degree"

## quantify the data

In [ ]:
quantified = data.pint.quantify()
quantified

## work with the data

In [ ]:
monthly_means = (
    quantified
    .pint.to("degC")
    .sel(time="2013")
    .groupby("time.month").mean()
)
monthly_means

## plot

`xarray`'s plotting functions will cast the data to `numpy.ndarray`, so we need to "dequantify" first.

In [ ]:
monthly_means.pint.dequantify(format="~P").plot.imshow(col="month", col_wrap=4)